<img src="https://www.cecodes.org.co/site/wp-content/uploads/2016/01/WP_asociado_COLSUBSIDIO-832x350.jpg" alt="Logo Colsubsidio" width=200>

# Modelo de Propensión a uso de servicios Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('../BD/Depuradas/BD.csv',index_col=False, encoding="latin1")
data.shape

(808553, 31)

In [3]:
data.columns.values

array(['id_afiliado', 'Futbol', 'Juegos', 'Fisico', 'Yoga', 'Infantil',
       'N_Beneficiarios', 'piramide1', 'piramide2', 'salario',
       'categoria', 'total_numero_grupo_familiar', 'genero', 'cx', 'cy',
       'nivel_socioeconomico', 'segmento_poblacional', 'marca', 'Edad',
       'descripcionciiu', 'actividad', 'n_afiliados', 'n_basico',
       'n_medio', 'n_joven', 'n_alto', 'n_f', 'n_m', 'n_categoriaa',
       'n_categoriab', 'n_categoriac'], dtype=object)

In [48]:
pd.value_counts(data.segmento_poblacional).to_frame().reset_index()

,index,segmento_poblacional
0,3,409846
1,5,362513
2,1,28854
3,2,7340


In [4]:
categoricas=["piramide1","piramide2","categoria","genero","descripcionciiu","actividad"]

for var in categoricas:
    data[var] = data[var].astype("category")

In [5]:
features=[
       'piramide1', 'piramide2', 'salario', 'categoria',
       'total_numero_grupo_familiar', 'genero',
       'Edad', 'descripcionciiu', 'actividad',
       'n_afiliados', 'n_basico', 'n_medio', 'n_joven', 'n_alto', 'n_f', 'n_m',
       'n_categoriaa', 'n_categoriab', 'n_categoriac']

targets=['Futbol','Juegos', 'Fisico', 'Yoga','Infantil']

X=data[features]
y=data[targets]

In [6]:
X=pd.get_dummies(X, columns=categoricas, prefix=["pir1", "pir2", "cate", "genero","CIIU","Act"])
X.head(3)

,salario,total_numero_grupo_familiar,Edad,n_afiliados,n_basico,n_medio,n_joven,n_alto,n_f,n_m,...,"Act_Agricultura, Caza, Silvicultura y pesca",Act_Comercio al por Mayor y al por Menor,"Act_Construccion, demoliciones, terrenos, Vias.","Act_Electricidad, Gas, Agua y explotacion de minas",Act_Industria,Act_Otros,"Act_Prestacion de Servicios (Educacion, Banca, Comunicaciones, Seguros, Salud, Arte y Cultura)",Act_Publico,Act_Tansporte y Almacenamiento,Act_Temporales
0,800000.0,2,30,2342,1809,291,242,3,1826,516,...,0,0,0,0,0,0,1,0,0,0
1,781242.0,1,25,2639,2245,291,102,1,2107,532,...,0,0,0,0,0,0,0,0,1,0
2,1777480.0,2,28,2170,1,2031,138,3,1014,1156,...,0,0,0,0,0,0,0,1,0,0


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31415)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(565987, 519) (242566, 519) (565987, 5) (242566, 5)


In [35]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=20, random_state=31415, 
                                                 class_weight='balanced'))

In [36]:
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=31415, verbose=0, warm_start=False),
          n_jobs=1)

In [37]:
y_pred = clf.predict_proba(X_test)

In [38]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred, average='macro')

0.9127561067533787

In [39]:
y_pred

array([[0.2565704 , 0.2574864 , 0.21911205, 0.37926165, 0.2021449 ],
       [0.18318967, 0.25585973, 0.12469914, 0.23437109, 0.23187326],
       [0.21466966, 0.18775863, 0.09332054, 0.18966579, 0.19369962],
       ...,
       [0.15232847, 0.11334595, 0.05402118, 0.20308881, 0.18132058],
       [0.16452822, 0.14867923, 0.08750828, 0.17107853, 0.22589738],
       [0.22778811, 0.24682846, 0.09866806, 0.23242476, 0.43161926]])

In [40]:
y_pred_t = clf.predict_proba(X_train)
roc_auc_score(y_train, y_pred_t, average='macro')

0.9847398965641295

In [41]:
y_pred_bd = clf.predict_proba(X)
roc_auc_score(y, y_pred_bd, average='macro')

0.9632091065861893

In [42]:
y_pred_bd

array([[0.10361054, 0.06887037, 0.04939748, 0.10231354, 0.14386535],
       [0.07170422, 0.01993994, 0.01870636, 0.0471656 , 0.12160452],
       [0.32918971, 0.08526598, 0.13942399, 0.26078667, 0.14625563],
       ...,
       [0.1301817 , 0.09015219, 0.08959467, 0.12753872, 0.28885281],
       [0.12256863, 0.10700179, 0.10216592, 0.14462434, 0.25741599],
       [0.18537066, 0.16402852, 0.28201859, 0.28622612, 0.25386523]])

In [43]:
Data_Fin = pd.concat([data.reset_index(drop=True), pd.DataFrame(y_pred_bd)], axis=1)

In [44]:
Data_Fin.head(10)

,id_afiliado,Futbol,Juegos,Fisico,Yoga,Infantil,N_Beneficiarios,piramide1,piramide2,salario,...,n_f,n_m,n_categoriaa,n_categoriab,n_categoriac,0,1,2,3,4
0,CC1000000210,0,0,0,0,0,3,2 Emp Medio,2.2 Silver,800000.0,...,1826,516,2219,102,21,0.103611,0.068870,0.049397,0.102314,0.143865
1,CC1000000255,0,0,0,0,0,2,3 Empresas Pymes,3.1 VIP,781242.0,...,2107,532,2572,58,9,0.071704,0.019940,0.018706,0.047166,0.121605
2,CC1000000381,0,0,0,0,0,3,1 Emp Grandes,1.1 Platinum,1777480.0,...,1014,1156,34,666,1470,0.329190,0.085266,0.139424,0.260787,0.146256
3,CC1000000418,0,0,0,0,0,2,3 Empresas Pymes,3.1 VIP,781242.0,...,1991,555,2437,96,13,0.036197,0.018229,0.026910,0.083765,0.101776
4,CC1000000426,0,0,0,0,0,3,1 Emp Grandes,1.2 Premium,815000.0,...,4214,2945,6322,756,81,0.088184,0.040771,0.029055,0.076291,0.064160
5,CC1000000456,0,0,0,0,0,4,1 Emp Grandes,1.2 Premium,781242.0,...,2711,3839,6084,406,60,0.120863,0.046862,0.026957,0.096781,0.090033
6,CC1000001427,0,0,0,0,0,3,1 Emp Grandes,1.1 Platinum,898704.0,...,4991,4716,8099,1333,275,0.026023,0.056460,0.020488,0.083765,0.039454
7,CC1000001437,0,0,0,0,0,2,2 Emp Medio,2.2 Silver,781242.0,...,324,1237,1424,115,22,0.087573,0.073765,0.037968,0.087945,0.107620
8,CC1000001511,0,0,0,0,0,3,2 Emp Medio,2.2 Silver,737717.0,...,324,1237,1424,115,22,0.117649,0.103790,0.038695,0.084234,0.146965
9,CC1000001514,0,0,0,0,0,3,3 Empresas Pymes,3.1 VIP,1550000.0,...,208,53,211,23,27,0.137006,0.134570,0.066993,0.117802,0.288880


In [46]:
Data_Fin.to_csv('../BD/Depuradas/BD_Calif.csv', sep='|', encoding='utf-8', index=False)

In [47]:
from sklearn.externals import joblib
joblib.dump(clf, '../Resultados/ModeloDados.pkl')

['../Resultados/ModeloDados.pkl']